In [1]:
#General libraries
import re
import os
import json

#Fine tune libraries
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments

#inference library
from transformers import GPT2LMHeadModel, GPT2Tokenizer 

In [11]:
#converting json to txt
prompt_json = r"C:\Users\HP\naibot\Sentiment_data\sentiment_data.json"
with open(prompt_json, mode='r', encoding="latin1") as prompts:
    df = json.load(prompts)
    
prompt_txt = []
for conversations in df:
    text = conversations['text']
    sentiment = conversations['sentiment']
    prompt_txt.append(text)
    prompt_txt.append(sentiment)

prompt_txt_file = r"C:\Users\HP\naibot\Sentiment_data\sentiment_data.txt"
with open(prompt_txt_file, 'w') as prompt_txt_file:
    for line in prompt_txt:
        try:
            prompt_txt_file.write(line)
            prompt_txt_file.write('\n')
        except:
            pass
print("Sentiment are already written to txt file successfully!")

Sentiment are already written to txt file successfully!


In [23]:
def read_documents_from_directory(directory):
    combined_text = ""
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        if filename.endswith(".txt"):
            with open(file_path, mode="r") as file:
                text = file.read()
                combined_text += text
    return combined_text

# Read documents from the directory
train_directory = r'C:\Users\HP\naibot\Sentiment_data'
text_data = read_documents_from_directory(train_directory)
text_data = re.sub(r'\n+', '\n', text_data).strip()  # Remove excess newline characters

print(text_data)

What a great day!!! Looks like dream.
positive
I feel sorry, I miss you here in the sea beach
positive
Don't angry me
negative
We attend in the class just for listening teachers reading on slide. Just Nonsence  
negative
Those who want to go, let them go
negative
Its night 2 am, feeling neutral
neutral
2 am feedings for the baby are fun when he is all smiles and coos
positive
Soooo high
neutral
 Both of you
neutral
Today first time I arrive in the boat. Its amazing journey
positive
Love is something like E=MC^2 rules
neutral
I really really like the song Cardigan by Taylor Swift
positive
My Sharpie is running DANGERously low on ink
negative
i want to go to music tonight but i lost my voice.
negative
Always somewhere I miss you where I have been
neutral
Don't distrube me
negative
I don't care, who are you and what can you do
negative
i`ve been sick for the past few days 
neutral
I'm come back home, my past job, its really good time for me
positive
I will never fall in love until I found

In [25]:
Train_directory = r"C:\Users\HP\naibot\Sentiment_data\train.txt"
with open(Train_directory, mode = "w") as train_data:
    train_data.write(text_data)
print("Successfully created train data!")

Successfully created train data!


In [26]:
# Define the function to load and tokenize the dataset
def load_dataset(file_path, tokenizer, block_size = 128):
    dataset = TextDataset(
        tokenizer = tokenizer,
        file_path = file_path,
        block_size = block_size,
    )

    return dataset

In [28]:
import os
import chardet
import re

def read_documents_from_directory(directory):
    combined_text = ""
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        if filename.endswith(".txt"):
            with open(file_path, 'rb') as file:
                raw_data = file.read()
                result = chardet.detect(raw_data)
                encoding = result['encoding']
                confidence = result['confidence']
                print(f"Detected encoding for {filename}: {encoding} with confidence {confidence}")

            # Attempt to open with detected encoding, fall back to 'latin1' if an error occurs
            try:
                with open(file_path, 'r', encoding=encoding) as file:
                    text = file.read()
            except UnicodeDecodeError:
                print(f"Failed to decode {filename} with encoding {encoding}. Trying 'latin1' instead.")
                with open(file_path, 'r', encoding='latin1', errors='ignore') as file:
                    text = file.read()

            combined_text += text

    return combined_text

# Read documents from the directory
train_directory = r'C:\Users\HP\naibot\Sentiment_data'
text_data = read_documents_from_directory(train_directory)
text_data = re.sub(r'\n+', '\n', text_data).strip()  # Remove excess newline characters

print(text_data)


Detected encoding for sentiment_data.txt: Windows-1252 with confidence 0.73
Detected encoding for train.txt: Windows-1252 with confidence 0.73
What a great day!!! Looks like dream.
positive
I feel sorry, I miss you here in the sea beach
positive
Don't angry me
negative
We attend in the class just for listening teachers reading on slide. Just Nonsence  
negative
Those who want to go, let them go
negative
Its night 2 am, feeling neutral
neutral
2 am feedings for the baby are fun when he is all smiles and coos
positive
Soooo high
neutral
 Both of you
neutral
Today first time I arrive in the boat. Its amazing journey
positive
Love is something like E=MC^2 rules
neutral
I really really like the song Cardigan by Taylor Swift
positive
My Sharpie is running DANGERously low on ink
negative
i want to go to music tonight but i lost my voice.
negative
Always somewhere I miss you where I have been
neutral
Don't distrube me
negative
I don't care, who are you and what can you do
negative
i`ve been si

In [29]:
# Load the data collator
def load_data_collator(tokenizer, mlm = False):
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, 
        mlm=mlm,
    )
    return data_collator

def train(train_file_path,model_name,
          output_dir,
          overwrite_output_dir,
          per_device_train_batch_size,
          num_train_epochs,
          save_steps):
  tokenizer = GPT2Tokenizer.from_pretrained(model_name)
  train_dataset = load_dataset(train_file_path, tokenizer)
  data_collator = load_data_collator(tokenizer)

  tokenizer.save_pretrained(output_dir)
  model = GPT2LMHeadModel.from_pretrained(model_name)

  model.save_pretrained(output_dir)

  training_args = TrainingArguments(
          output_dir=output_dir,
          overwrite_output_dir=overwrite_output_dir,
          per_device_train_batch_size=per_device_train_batch_size,
          num_train_epochs=num_train_epochs,
      )

  trainer = Trainer(
          model=model,
          args=training_args,
          data_collator=data_collator,
          train_dataset=train_dataset,
  )
      
  trainer.train()
  trainer.save_model()

# Define the parameters for training
train_file_path = r"C:\Users\HP\naibot\Sentiment_data\train.txt"
model_name = 'gpt2'
output_dir = r'C:\Users\HP\naibot\Sentiment_Finetuned_output'
overwrite_output_dir = True
per_device_train_batch_size = 2
num_train_epochs = 1
save_steps = 20000

# Training the model
train(
    train_file_path=train_file_path,
    model_name=model_name,
    output_dir=output_dir,
    overwrite_output_dir=overwrite_output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    num_train_epochs=num_train_epochs,
    save_steps=save_steps,  
)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x92 in position 10509: invalid start byte